In [1]:
from agents.dpm__agent import Agent, preprocess, test_reward
from trading_env.environment import TradingEnv

In [2]:
import dill
import numpy as np
import tensorflow as tf

In [3]:
yf_file = "./data/archive_data/yf_data.dill"
with open(yf_file,'rb') as dill_file:
    yf_df = dill.load(dill_file)
    

In [4]:
symbols = yf_df['Symbol'].unique()
stocks = []
for symbol in symbols:
    df = yf_df[yf_df['Symbol'] == symbol]
    stocks.append(df.iloc[:,2:6].to_numpy())
stocks = np.array(stocks)

In [5]:
n_stocks = stocks.shape[0]
window = 64
n_stock_feats = stocks.shape[2]

In [6]:
agent = Agent(n_stocks,n_stock_feats)

In [7]:
env = TradingEnv(stocks,training_size=0.1)
obs = env.reset()
done = False
while not done:
    action = agent.act(obs) 
    obs, reward, done, _ = env.step(action) 
print('Untrained portfolio end value:')
print(env.portfolio_value_hist[-1])


Untrained portfolio end value:
tf.Tensor(1027114.3, shape=(), dtype=float32)


In [8]:
def loss_val():
    total_loss = tf.convert_to_tensor(0.0)
    env = TradingEnv(stocks,training_size=0.1)
    obs = env.reset()
    done = False
    while not done:

        action = agent.act(obs)
        obs,reward,done,_=env.step(action)
        total_loss -= reward

    return total_loss, env



In [9]:
portfolio_time_series = []
print('Portfolio value after n gradient ascent steps:')
for _ in range(5):
    with tf.GradientTape() as tape:
        tape.watch(agent.model.trainable_variables)
        loss, env = loss_val()
    portfolio_time_series.append(env.portfolio_value_hist)
    print(env.portfolio_value_hist[-1])
    grad = tape.gradient(loss,agent.model.trainable_variables)
    agent.opt.apply_gradients(zip(grad,agent.model.trainable_variables))
loss, env = loss_val()
print(env.portfolio_value_hist[-1])

Portfolio value after n gradient ascent steps:
tf.Tensor(1027114.3, shape=(), dtype=float32)
tf.Tensor(1022618.1, shape=(), dtype=float32)
tf.Tensor(1071041.5, shape=(), dtype=float32)
tf.Tensor(1237350.4, shape=(), dtype=float32)
tf.Tensor(1314853.1, shape=(), dtype=float32)
tf.Tensor(1422374.4, shape=(), dtype=float32)
